In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = "/kaggle/input/jane-street-market-prediction/train.csv"
df_train = dt.fread(path ,fill = True)
df_train = df_train.to_pandas()
df_train = df_train.query('date > 85').reset_index(drop = True) 
df_train = df_train[df_train['weight'] != 0]
df_train.fillna(df_train.mean(),inplace=True)
df_train['action'] = (df_train.resp > 0).astype(int)


In [ ]:
features_mask = ['feature_'+str(i) for i in range(0,130)]
features_mask.append('weight')
ret_mask = ['resp_4','resp_3','resp_2','resp_1','resp']

In [ ]:
X = df_train[features_mask]
y = df_train['resp']

In [ ]:
sample_size = 2000
time_step = 4

In [ ]:
sample_size = 2000
time_step = 4
X_sub = X[:sample_size]
y_sub = y[time_step:+sample_size+time_step]
n_features = 131
X_sub = X_sub.to_numpy()

def transfer(time_step,X_sub):
    X_p = []
    sample_size = len(X_sub)
    for i in range(sample_size-time_step+1):
        l = []
        for j in range(time_step):
            l.append(X_sub[i+j])
        X_p.append(l)
    X_p = np.array(X_p)
    return X_p

X_train = transfer(time_step, X_sub)[:int(sample_size*0.75)]
y_train = y_sub[:int(sample_size*0.75)]


In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.activations import *
from tensorflow.keras import *
import tensorflow as tf

In [ ]:
#tf.config.run_functions_eagerly(True)

#model = Sequential()
#model.add(LSTM(140, input_shape=(time_step, n_features),return_sequences=True, activation = tf.keras.activations.relu))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))    
#model.add(LSTM(40,activation = tf.keras.activations.swish))
#model.add(Dense(1, activation = sigmoid))

#model.compile(optimizer='adam', loss='mse')

In [ ]:
length = 4
inp= Input(shape=(length,131))
x = LSTM(160,return_sequences=True)(inp)
x = tf.keras.layers.Dense(1)(x)

x = LSTM(160,return_sequences=True)(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = LSTM(40,)(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Dense(1)(x)
out = tf.keras.layers.Activation("sigmoid")(x)

model = tf.keras.models.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, y_train,epochs=10, verbose=0,batch_size=500)

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

i = 0
lst = []

for (test_df, sample_prediction_df) in iter_test:
    if i < 4:
        lst.append(test_df[features_mask].to_numpy())
        sample_prediction_df.action = 0
        i = i+1

    else:    
        lst = lst[1:]
        lst.append(test_df[features_mask].to_numpy())
        input_ = np.array(lst).reshape(1,4,131)
        if model.predict(input_,verbose = 0) > 0.025:
            sample_prediction_df.action = 1
        else:
             sample_prediction_df.action = 0
    env.predict(sample_prediction_df)